# APLICACIONES EN CIENCIAS DE COMPUTACION
Dr. Edwin Villanueva Talavera (ervillanueva@pucp.edu.pe)

## Practica Guiada:  Busqueda local para solucionar Tableros de Sudoku

Este notebook implementa busqueda local para encontrar soluciones a tableros de sudoku. Se les esta disponibilizando la implementacion del algoritmo Simulating Annealing y Hill climbing. Al final de este notebook estan las preguntas 


### Funciones utilitarias para manejar el tablero del sudoku </b>
Estas son funciones utilitarias para trabajar con tableros de sudoku  

In [6]:
import sys
import time
import numpy as np
from random import shuffle, random, sample, randint
from copy import deepcopy
from math import exp

# Funcion para cargar el tablero del sudoku de un archivo
def readBoardFromFile(filename):
    fd = open(filename,"r+")    
    puzzle = eval(fd.readline())
    board = []
    for row in puzzle:
        for col in row:
            board.append(col)
    return np.array(board)  #  el tablero es un vector con las filas concatenadas

# devuelve los indices de los elementos de la columna i del tablero  
def get_column_indices(i):  
    indices = [i + 9 * j for j in range(9)]
    return indices

# devuelve los indices de los elementos de la fila i del tablero 
def get_row_indices(i):  
    indices = [j + 9*i for j in range(9)]
    return indices

# devuelve los indices de los elementos del bloque k del tablero. initialEntries son los indices de las posiciones inmutables (numeros iniciales) 
# si ignore_originals=true  entonces solo devuelve los indices de las posiciones que no son las posiciones inmutables
def get_block_indices(k, initialEntries, ignore_originals=False): 
    row_offset = (k//3)*3
    col_offset= (k%3)*3
    indices=[col_offset+(j%3)+9*(row_offset+(j//3)) for j in range(9)]
    if ignore_originals:
        indices = [x for x in indices if x not in initialEntries]
    return indices

# Completa aleatoriamente las pociciones vacias (que no son initialEntries). 
# Garantiza que en cada bloque se generen 9 numeros diferentes 
def randomAssign(board, initialEntries):
    for num in range(9):
        block_indices = get_block_indices(num, initialEntries)
        block = board[block_indices]
        zero_indices=[ind for i,ind in enumerate(block_indices) if block[i] == 0]
        to_fill = [i for i in range(1,10) if i not in block]
        shuffle(to_fill)
        for ind, value in zip(zero_indices, to_fill):
            board[ind] = value

# Evalua un tablero. Devuelve la cantidad de numeros diferentes en todas las filas y columnas 
# un tablero solucion tiene un score de 162 (81+81)
def score_board(board):
    score = 0
    for row in range(9): # por cada fila obtiene la cantidad de numeros diferentes
        score+= len(set(board[get_row_indices(row)]))
    for col in range(9): # por cada columna obtiene la cantidad de numeros diferentes
        score += len(set(board[get_column_indices(col)]))
    return score

# Genera un tablero nuevo. Se escoje aleatoriamente un bloque y se intercambia dos valores (no dados en el tablero original)  
def make_neighborBoard(board, initialEntries):
    new_board = deepcopy(board)
    block = randint(0,8)  # escoje un bloque aleatoriamente
    num_in_block = len(get_block_indices(block,initialEntries,ignore_originals=True)) #cantidad de posiciones que se puede alterar en el bloque 
    random_squares = sample(range(num_in_block),2) # escoje dos posiciones aleatorias del bloque para intercambiar valores
    square1, square2 = [get_block_indices(block,initialEntries,ignore_originals=True)[ind] for ind in random_squares]
    new_board[square1], new_board[square2] = new_board[square2], new_board[square1] # intercambia los valores de las posiciones
    return new_board

# Imprime un tablero en pantalla
def showBoard(board):
    def checkZero(s):
        if s != 0: return str(s)
        if s == 0: return "0"
    results = np.array([board[get_row_indices(j)] for j in range(9)])
    s=""
    for i, row in enumerate(results):
        if i%3==0:
            s +="-"*25+'\n'
        s += "| " + " | ".join([" ".join(checkZero(s) for s in list(row)[3*(k-1):3*k]) for k in range(1,4)]) + " |\n"
    s +="-"*25+''
    print (s)

### Implementacion del algoritmo Hill Climbing  para resolver tableros de sudoku </b>


In [7]:
def get_neighborhood(board, initialEntries, number_of_neighbors=100):
    """ finds neighboring boards using make_neighborBoard() method """
    neighborhood = []
    for i in range(number_of_neighbors):
        neighborhood.append(make_neighborBoard(board, initialEntries))
    return neighborhood


def hc_solver(board, number_of_neighbors=100, maxIter=1000 ):
    """ Hill Climbing solver. From the initial board, keep choosing the neighbor board with highest score,
        stopping when no neighbor is better. 
        board: is a np array of 81 elements. The first 9 are the first row of the board, the next 9 are the second row ...            
        number_of_neighbors: Number of neighbors in the neighborhood
        maxIter: The maximum number of iterations
    """
    start_time = time.time()
    print ('Hill Climbing intentará resolver el siguiente tablero sudoku: ')
    showBoard(board)
    initialEntries = np.arange(81)[board > 0]  # las posiciones no vacias del board
    randomAssign(board, initialEntries)  # En cada box del board asigna numeros aleatorios en pociciones vacias, garantizando que sean los 9 numeros diferentes 
    
    current_board = board
    current_score = score_board(board)
    t = 0
    while (t < maxIter):  # bucle principal
        if (t % 100 == 0): 
            print('Iteration {},\tCurrent score = {}'.format(t, current_score))
        
        neighborhood = get_neighborhood(current_board, initialEntries, number_of_neighbors)  # obtiene tableros vecinos 
        if not neighborhood:
            break
        
        neighborhood_scores = []
        for i in range(len(neighborhood)): # evaluate each neighbor board
            neighborhood_scores.append( score_board(neighborhood[i]) )
        index_best_neighbor = np.argmax(neighborhood_scores)  # the index of the best neighbor
            
        if neighborhood_scores[index_best_neighbor] >= current_score:
            current_score = neighborhood_scores[index_best_neighbor]
            current_board = deepcopy(neighborhood[index_best_neighbor])
        t += 1
    
    end_time = time.time()
    print("Best board found in {} seconds ({} ierations) with {}/{} points:".format(end_time-start_time,t,current_score,162))
    showBoard(current_board)

### Implementacion del algoritmo Simulating Annealing  para resolver tableros de sudoku </b>


In [8]:
def sa_solver(board, T0=0.5, DR=.99999, maxIter=100000):
    """ Simulating annealing solver. 
        board: is a np array of 81 elements. The first 9 are the first row of the board, the next 9 are the second row ...    
           T0: Initial temperature
           DR: The decay rate of the schedule function: Ti = T0*(DR)^i (Ti is the temperature at iteration i). 
               For efficiecy the schedule function is implemented as: Ti = T(i-1)*DR
      maxIter: The maximum number of iterations
    """
    start_time = time.time()
    print ('Simulated Annealing intentará resolver el siguiente tablero sudoku: ')
    showBoard(board)
    
    initialEntries = np.arange(81)[board > 0]  # las posiciones no vacias del board
    randomAssign(board, initialEntries)  # En cada box del board asigna numeros aleatorios en pociciones vacias, garantizando que sean los 9 numeros diferentes 
    #showBoard(board)
    best_board = deepcopy(board)
    current_score = score_board(board)
    best_score = current_score
    T = T0  # El valor inicial de la temperatura
    t = 0
    while (t < maxIter):
        try:
            if (t % 10000 == 0): 
                print('Iteration {},\tTemperaure = {},\tBest score = {},\tCurrent score = {}'.format(t, T, best_score, current_score))
            
            neighborBoard = make_neighborBoard(board, initialEntries)  # obtiene un tablero vecino
            neighborBoardScore = score_board(neighborBoard)  # evalua tablero vecino
            delta = float(neighborBoardScore - current_score)
            
            if (delta > 0):   # si el tablero vecino generado es mejor que el actual, se acepta
                board = neighborBoard
                current_score = neighborBoardScore 
            
            elif (exp((delta/T)) > random()):  #si el tablero generado es peor, se acepta con prob exp((delta/T))
                board = neighborBoard
                current_score = neighborBoardScore 
            
            
            if (current_score > best_score):  # si el tablero actual es mejor que el actual mejor
                best_board = deepcopy(board)
                best_score = score_board(best_board)
                
            if neighborBoardScore == 162:   # 162 es un tablero solucion
                best_board = neighborBoard
                break

            T = DR*T   # la nueva temperatura decae con factor de decaimiento, es una forma eficiente del schedule Ti = T0*(DR)^i
            t += 1
        except:
            print("Numerical error occurred. It's a random algorithm so try again.")         
    end_time = time.time() 
    if best_score == 162:
        print ("Solution found in {} seconds (iterations={}):".format(end_time - start_time, t))
    else:
        print("Couldn't solve! ({}/{} points). This is the best solution found (iterations={}):".format(best_score,162,t))
    
    showBoard(best_board)

## <b> Experimentación con los algoritmos de Busqueda</b> 


Llama a Hill Climbing para resolver el tablero 'puzzleA.txt' con number_of_neighbors=100 y numero de iteraciones maxIter=1000

In [9]:
board = readBoardFromFile('puzzleA.txt')
hc_solver(board, number_of_neighbors=100, maxIter=1000)

Hill Climbing intentará resolver el siguiente tablero sudoku: 
-------------------------
| 0 0 1 | 4 0 0 | 3 8 0 |
| 0 0 0 | 6 3 0 | 1 0 0 |
| 0 3 8 | 0 0 0 | 0 7 0 |
-------------------------
| 0 0 3 | 0 0 1 | 0 9 0 |
| 0 6 0 | 0 7 3 | 0 0 0 |
| 0 0 0 | 0 0 0 | 4 0 0 |
-------------------------
| 0 0 0 | 0 0 0 | 0 4 0 |
| 2 1 0 | 0 6 0 | 0 0 0 |
| 4 0 0 | 9 0 0 | 7 0 0 |
-------------------------
Iteration 0,	Current score = 121
Iteration 100,	Current score = 156
Iteration 200,	Current score = 156
Iteration 300,	Current score = 156
Iteration 400,	Current score = 156
Iteration 500,	Current score = 156
Iteration 600,	Current score = 156
Iteration 700,	Current score = 156
Iteration 800,	Current score = 156
Iteration 900,	Current score = 156
Best board found in 24.062796354293823 seconds (1000 ierations) with 156/162 points:
-------------------------
| 6 2 1 | 4 5 7 | 3 8 9 |
| 9 5 7 | 6 3 8 | 1 2 4 |
| 4 3 8 | 1 9 2 | 6 7 5 |
-------------------------
| 5 4 3 | 2 6 1 | 8 9 7 |
| 8 6 9 | 

Llama a Simulating Annealing para resolver el tablero 'puzzleA.txt' con Temperatura inicial T0=0.5, Factor de decaimiento de temperatura DR=0.99999 y Máximo numero de iteraciones maxIter=100000

In [14]:
board = readBoardFromFile('puzzleA.txt')
sa_solver(board, T0=0.5, DR=.999999, maxIter=200000)

Simulated Annealing intentará resolver el siguiente tablero sudoku: 
-------------------------
| 0 0 1 | 4 0 0 | 3 8 0 |
| 0 0 0 | 6 3 0 | 1 0 0 |
| 0 3 8 | 0 0 0 | 0 7 0 |
-------------------------
| 0 0 3 | 0 0 1 | 0 9 0 |
| 0 6 0 | 0 7 3 | 0 0 0 |
| 0 0 0 | 0 0 0 | 4 0 0 |
-------------------------
| 0 0 0 | 0 0 0 | 0 4 0 |
| 2 1 0 | 0 6 0 | 0 0 0 |
| 4 0 0 | 9 0 0 | 7 0 0 |
-------------------------
Iteration 0,	Temperaure = 0.5,	Best score = 113,	Current score = 113
Iteration 10000,	Temperaure = 0.4950249143993146,	Best score = 158,	Current score = 154
Iteration 20000,	Temperaure = 0.49009933175209747,	Best score = 158,	Current score = 153
Iteration 30000,	Temperaure = 0.48522275949548915,	Best score = 158,	Current score = 149
Iteration 40000,	Temperaure = 0.48039470996770894,	Best score = 158,	Current score = 153
Iteration 50000,	Temperaure = 0.47561470035929837,	Best score = 158,	Current score = 151
Iteration 60000,	Temperaure = 0.4708822526648362,	Best score = 160,	Current scor